# Pytorch Quick Intro

In [1]:
import torch
print(torch.__version__)

2.1.2


## Quick code examples

## Before Pytorch 2.0

In [2]:
import torch
import torchvision

model = torchvision.models.resnet50()

## After Pytorch 2.0

In [3]:
import torch 
import torchvision

model = torchvision.models.resnet50()
compiled_model = torch.compile(model)

## 1. Get GPU info

In [3]:
# Make sure we're using a NVIDIA GPU
if torch.cuda.is_available():
  gpu_info = !nvidia-smi
  gpu_info = '\n'.join(gpu_info)
  if gpu_info.find("failed") >= 0:
    print("Not connected to a GPU, to leverage the best of PyTorch 2.0, you should connect to a GPU.")

  # Get GPU name
  gpu_name = !nvidia-smi --query-gpu=gpu_name --format=csv
  gpu_name = gpu_name[1]
  GPU_NAME = gpu_name.replace(" ", "_") # remove underscores for easier saving
  print(f'GPU name: {GPU_NAME}')

  # Get GPU capability score
  GPU_SCORE = torch.cuda.get_device_capability()
  print(f"GPU capability score: {GPU_SCORE}")
  if GPU_SCORE >= (8, 0):
    print(f"GPU score higher than or equal to (8, 0), PyTorch 2.x speedup features available.")
  else:
    print(f"GPU score lower than (8, 0), PyTorch 2.x speedup features will be limited (PyTorch 2.x speedups happen most on newer GPUs).")
  
  # Print GPU info
  print(f"GPU information:\n{gpu_info}")

else:
  print("PyTorch couldn't find a GPU, to leverage the best of PyTorch 2.0, you should connect to a GPU.")

GPU name: Tesla_T4
GPU capability score: (7, 5)
GPU score lower than (8, 0), PyTorch 2.x speedup features will be limited (PyTorch 2.x speedups happen most on newer GPUs).
GPU information:
Wed Jun 12 10:12:27 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8               9W /  70W |      3MiB / 153

### 1.1 Globally set device

In [7]:
import torch 

device = "cuda" if torch.cuda.is_available() else "cpu"


# Set the device with context manager(PyTorch 2.x+)
with torch.device(device):
    layer = torch.nn.Linear(20, 30)
    print(f"layer weights are on device: {layer.weight.device}")
    print(f"layer creating data on device: {layer(torch.randn(128, 20)).device}")

layer weights are on device: cuda:0
layer creating data on device: cuda:0


In [10]:
import torch 

device = "cuda" if torch.cuda.is_available() else "cpu"

# Set the device globally (Pytorch 2.x+)
torch.set_default_device(device)

layer = torch.nn.Linear(20, 30)
print(f"layer weights are on device: {layer.weight.device}")
print(f"layer creating data on device: {layer(torch.randn(128, 20)).device}")


layer weights are on device: cuda:0
layer creating data on device: cuda:0


## 2. Setting up the experiments

In [11]:
import torch
import torchvision

print(f"PyTorch version: {torch.__version__}")
print(f"TorchVision version: {torchvision.__version__}")

# Set the target device
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Using device: {device}")

PyTorch version: 2.1.2
TorchVision version: 0.16.2
Using device: cuda


In [12]:
torch.set_default_device(device)

### 2.1 Create model and transforms

In [18]:

def create_resnet50(num_classes:int=10):
    weights = torchvision.models.ResNet50_Weights.DEFAULT
    transforms = weights.transforms
    model = torchvision.models.resnet50(weights=weights)
    model.fc = torch.nn.Linear(in_features=2048,
                         out_features=num_classes)

    
    return model, transforms
    
model, transforms = create_resnet50()
transforms

functools.partial(<class 'torchvision.transforms._presets.ImageClassification'>, crop_size=224, resize_size=232)

In [14]:
total_params = sum(
param.numel() for param in model.parameters()
)
total_params

25557032